<a href="https://colab.research.google.com/github/luzalbaposse/IntroduccionABlockchain/blob/main/Trabajo_Pr%C3%A1ctico_N%C3%BAmero_2_Introducci%C3%B3n_a_Blockchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 👉 Trabajo Práctico 2: Smart Contracts
El objetivo es leer y comprender la estructura de smart contracts que posee el protocolo elegido previamente para el TP1 (en este caso, MakerDAO), agregando además la utilización de librerías en **Python** o Javascript para poder obtener información en vivo de los protocolos.

## 💻 Miembros: Mateo Clutterbuck & Luz Alba Posse



# 1. ¿Cuál es la estructura core de smart contracts de este protocolo?


El **core module** de MakerDAO está compuesto por Vat.sol y Spot.sol, contienen todo el estado de Maker y controla los mecanismos centrales del sistema mientras se encuentra en el estado esperado de funcionamiento.

A grandes rasgos, en vat.sol está el estado del Vault, dai y sus colaterales. No tiene dependencias externas pero mantiene las invariantes contables centrales de DAI. Después, en spot.sol hay una función que es la mas relevante: poke. En general, toma en uno bytes32 de los ilk que van a ser "poked", llamando a dos funciones externas que son peek y file.

Un detalle relevante es que quienes gobiernan MakerDAO pueden autorizar nuevos módulos frente al Vat, entonces se puee robar garantías (slip) o tener Dai sin respaldo (suck).


## 🪙 DAI

### 1. dai.sol -- DAI Stablecoin ERC-20 Token

In [ ]:
// SPDX-License-Identifier: AGPL-3.0-or-later

/// dai.sol -- Dai Stablecoin ERC-20 Token

// Copyright (C) 2017, 2018, 2019 dbrock, rain, mrchico

// This program is free software: you can redistribute it and/or modify
// it under the terms of the GNU Affero General Public License as published by
// the Free Software Foundation, either version 3 of the License, or
// (at your option) any later version.
//
// This program is distributed in the hope that it will be useful,
// but WITHOUT ANY WARRANTY; without even the implied warranty of
// MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
// GNU Affero General Public License for more details.
//
// You should have received a copy of the GNU Affero General Public License
// along with this program.  If not, see <https://www.gnu.org/licenses/>.

pragma solidity ^0.6.12;

// FIXME: This contract was altered compared to the production version.
// It doesn't use LibNote anymore.
// New deployments of this contract will need to include custom events (TO DO).

contract Dai {
    // --- Auth ---
    mapping (address => uint) public wards;
    function rely(address guy) external auth { wards[guy] = 1; }
    function deny(address guy) external auth { wards[guy] = 0; }
    modifier auth {
        require(wards[msg.sender] == 1, "Dai/not-authorized");
        _;
    }

    // --- ERC20 Data ---
    string  public constant name     = "Dai Stablecoin";
    string  public constant symbol   = "DAI";
    string  public constant version  = "1";
    uint8   public constant decimals = 18;
    uint256 public totalSupply;

    mapping (address => uint)                      public balanceOf;
    mapping (address => mapping (address => uint)) public allowance;
    mapping (address => uint)                      public nonces;

    event Approval(address indexed src, address indexed guy, uint wad);
    event Transfer(address indexed src, address indexed dst, uint wad);

    // --- Math ---
    function add(uint x, uint y) internal pure returns (uint z) {
        require((z = x + y) >= x);
    }
    function sub(uint x, uint y) internal pure returns (uint z) {
        require((z = x - y) <= x);
    }

    // --- EIP712 niceties ---
    bytes32 public DOMAIN_SEPARATOR;
    // bytes32 public constant PERMIT_TYPEHASH = keccak256("Permit(address holder,address spender,uint256 nonce,uint256 expiry,bool allowed)");
    bytes32 public constant PERMIT_TYPEHASH = 0xea2aa0a1be11a07ed86d755c93467f4f82362b452371d1ba94d1715123511acb;

    constructor(uint256 chainId_) public {
        wards[msg.sender] = 1;
        DOMAIN_SEPARATOR = keccak256(abi.encode(
            keccak256("EIP712Domain(string name,string version,uint256 chainId,address verifyingContract)"),
            keccak256(bytes(name)),
            keccak256(bytes(version)),
            chainId_,
            address(this)
        ));
    }

    // --- Token ---
    function transfer(address dst, uint wad) external returns (bool) {
        return transferFrom(msg.sender, dst, wad);
    }
    function transferFrom(address src, address dst, uint wad)
        public returns (bool)
    {
        require(balanceOf[src] >= wad, "Dai/insufficient-balance");
        if (src != msg.sender && allowance[src][msg.sender] != uint(-1)) {
            require(allowance[src][msg.sender] >= wad, "Dai/insufficient-allowance");
            allowance[src][msg.sender] = sub(allowance[src][msg.sender], wad);
        }
        balanceOf[src] = sub(balanceOf[src], wad);
        balanceOf[dst] = add(balanceOf[dst], wad);
        emit Transfer(src, dst, wad);
        return true;
    }
    function mint(address usr, uint wad) external auth {
        balanceOf[usr] = add(balanceOf[usr], wad);
        totalSupply    = add(totalSupply, wad);
        emit Transfer(address(0), usr, wad);
    }
    function burn(address usr, uint wad) external {
        require(balanceOf[usr] >= wad, "Dai/insufficient-balance");
        if (usr != msg.sender && allowance[usr][msg.sender] != uint(-1)) {
            require(allowance[usr][msg.sender] >= wad, "Dai/insufficient-allowance");
            allowance[usr][msg.sender] = sub(allowance[usr][msg.sender], wad);
        }
        balanceOf[usr] = sub(balanceOf[usr], wad);
        totalSupply    = sub(totalSupply, wad);
        emit Transfer(usr, address(0), wad);
    }
    function approve(address usr, uint wad) external returns (bool) {
        allowance[msg.sender][usr] = wad;
        emit Approval(msg.sender, usr, wad);
        return true;
    }

    // --- Alias ---
    function push(address usr, uint wad) external {
        transferFrom(msg.sender, usr, wad);
    }
    function pull(address usr, uint wad) external {
        transferFrom(usr, msg.sender, wad);
    }
    function move(address src, address dst, uint wad) external {
        transferFrom(src, dst, wad);
    }

    // --- Approve by signature ---
    function permit(address holder, address spender, uint256 nonce, uint256 expiry,
                    bool allowed, uint8 v, bytes32 r, bytes32 s) external
    {
        bytes32 digest =
            keccak256(abi.encodePacked(
                "\x19\x01",
                DOMAIN_SEPARATOR,
                keccak256(abi.encode(PERMIT_TYPEHASH,
                                     holder,
                                     spender,
                                     nonce,
                                     expiry,
                                     allowed))
        ));

        require(holder != address(0), "Dai/invalid-address-0");
        require(holder == ecrecover(digest, v, r, s), "Dai/invalid-permit");
        require(expiry == 0 || now <= expiry, "Dai/permit-expired");
        require(nonce == nonces[holder]++, "Dai/invalid-nonce");
        uint wad = allowed ? uint(-1) : 0;
        allowance[holder][spender] = wad;
        emit Approval(holder, spender, wad);
    }
}

Es un contrato ERC20 básicamente para usuarios que mantiene el registro de los saldos externos de DAI. La mayoría de las funciones son estándar para un token con suministro variable, pero destaca por la capacidad de emitir aprobaciones para transferencias basadas en mensajes firmados.

Hay algunas diferencias con el estándar ERC20:

1. `transferFrom` permite *aprobar de forma limitada*. O sea que si una persona aprueba una dirección para usar sus tokens en DAI por el valor máximo posible, esa dirección va a tener aprobación ilimitada hasta que se le indique lo contrario. Entonces, la dirección aprobada puede transferir tokens de la cuenta del usuario sin necesidad de que este apruebe cada una de las transacciones.
2. `push`, `pull` y `move` son simplemente alias para la función `transferFrom`, hacen exactamente lo mismo que la función `transferFrom` pero con diferentes nombres.
3. `permit` es una forma de **aprobación basada en la firma**. O sea que una persona puede firmar un **mensaje** que luego **puede ser utilizado por otra parte para enviar su aprobación**. Sirve para aplicaciones en las que el usuario final no necesita tener `ETH`.

    Para usar esto, la dirección de un usuario debe firmar un mensaje con el `holder` (dueño de los tokens), `spender` (persona u organización a la que se le autoriza usar los tokens), `nonce` (un número que se incrementa con cada transacción), `expiry` (un tiempo límite para que la transacción sea válida) y el monto `allowed` (cantidad máxima de tokens aprobados). Ahí, el mensaje se puede enviar a la función `Permit()` para actualizar la aprobación del usuario y permitir que la dirección realice transacciones en su nombre.

**Importante**: `wad` punto decimal fijo con 18 decimales (para cantidades básicas, por ejemplo, balanzas).

## ✨ Pot

In [ ]:
// SPDX-License-Identifier: AGPL-3.0-or-later

/// pot.sol -- Dai Savings Rate

// Copyright (C) 2018 Rain <rainbreak@riseup.net>
//
// This program is free software: you can redistribute it and/or modify
// it under the terms of the GNU Affero General Public License as published by
// the Free Software Foundation, either version 3 of the License, or
// (at your option) any later version.
//
// This program is distributed in the hope that it will be useful,
// but WITHOUT ANY WARRANTY; without even the implied warranty of
// MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
// GNU Affero General Public License for more details.
//
// You should have received a copy of the GNU Affero General Public License
// along with this program.  If not, see <https://www.gnu.org/licenses/>.

pragma solidity ^0.6.12;

// FIXME: This contract was altered compared to the production version.
// It doesn't use LibNote anymore.
// New deployments of this contract will need to include custom events (TO DO).

/*
   "Savings Dai" is obtained when Dai is deposited into
   this contract. Each "Savings Dai" accrues Dai interest
   at the "Dai Savings Rate".

   This contract does not implement a user tradeable token
   and is intended to be used with adapters.

         --- `save` your `dai` in the `pot` ---

   - `dsr`: the Dai Savings Rate
   - `pie`: user balance of Savings Dai

   - `join`: start saving some dai
   - `exit`: remove some dai
   - `drip`: perform rate collection

*/

interface VatLike {
    function move(address,address,uint256) external;
    function suck(address,address,uint256) external;
}

contract Pot {
    // --- Auth ---
    mapping (address => uint) public wards;
    function rely(address guy) external auth { wards[guy] = 1; }
    function deny(address guy) external auth { wards[guy] = 0; }
    modifier auth {
        require(wards[msg.sender] == 1, "Pot/not-authorized");
        _;
    }

    // --- Data ---
    mapping (address => uint256) public pie;  // Normalised Savings Dai [wad]

    uint256 public Pie;   // Total Normalised Savings Dai  [wad]
    uint256 public dsr;   // The Dai Savings Rate          [ray]
    uint256 public chi;   // The Rate Accumulator          [ray]

    VatLike public vat;   // CDP Engine
    address public vow;   // Debt Engine
    uint256 public rho;   // Time of last drip     [unix epoch time]

    uint256 public live;  // Active Flag

    // --- Init ---
    constructor(address vat_) public {
        wards[msg.sender] = 1;
        vat = VatLike(vat_);
        dsr = ONE;
        chi = ONE;
        rho = now;
        live = 1;
    }

    // --- Math ---
    uint256 constant ONE = 10 ** 27;
    function _rpow(uint x, uint n, uint base) internal pure returns (uint z) {
        assembly {
            switch x case 0 {switch n case 0 {z := base} default {z := 0}}
            default {
                switch mod(n, 2) case 0 { z := base } default { z := x }
                let half := div(base, 2)  // for rounding.
                for { n := div(n, 2) } n { n := div(n,2) } {
                    let xx := mul(x, x)
                    if iszero(eq(div(xx, x), x)) { revert(0,0) }
                    let xxRound := add(xx, half)
                    if lt(xxRound, xx) { revert(0,0) }
                    x := div(xxRound, base)
                    if mod(n,2) {
                        let zx := mul(z, x)
                        if and(iszero(iszero(x)), iszero(eq(div(zx, x), z))) { revert(0,0) }
                        let zxRound := add(zx, half)
                        if lt(zxRound, zx) { revert(0,0) }
                        z := div(zxRound, base)
                    }
                }
            }
        }
    }

    function _rmul(uint x, uint y) internal pure returns (uint z) {
        z = _mul(x, y) / ONE;
    }

    function _add(uint x, uint y) internal pure returns (uint z) {
        require((z = x + y) >= x);
    }

    function _sub(uint x, uint y) internal pure returns (uint z) {
        require((z = x - y) <= x);
    }

    function _mul(uint x, uint y) internal pure returns (uint z) {
        require(y == 0 || (z = x * y) / y == x);
    }

    // --- Administration ---
    function file(bytes32 what, uint256 data) external auth {
        require(live == 1, "Pot/not-live");
        require(now == rho, "Pot/rho-not-updated");
        if (what == "dsr") dsr = data;
        else revert("Pot/file-unrecognized-param");
    }

    function file(bytes32 what, address addr) external auth {
        if (what == "vow") vow = addr;
        else revert("Pot/file-unrecognized-param");
    }

    function cage() external auth {
        live = 0;
        dsr = ONE;
    }

    // --- Savings Rate Accumulation ---
    function drip() external returns (uint tmp) {
        require(now >= rho, "Pot/invalid-now");
        tmp = _rmul(_rpow(dsr, now - rho, ONE), chi);
        uint chi_ = _sub(tmp, chi);
        chi = tmp;
        rho = now;
        vat.suck(address(vow), address(this), _mul(Pie, chi_));
    }

    // --- Savings Dai Management ---
    function join(uint wad) external {
        require(now == rho, "Pot/rho-not-updated");
        pie[msg.sender] = _add(pie[msg.sender], wad);
        Pie             = _add(Pie,             wad);
        vat.move(msg.sender, address(this), _mul(chi, wad));
    }

    function exit(uint wad) external {
        pie[msg.sender] = _sub(pie[msg.sender], wad);
        Pie             = _sub(Pie,             wad);
        vat.move(address(this), msg.sender, _mul(chi, wad));
    }
}

Este contrato es responsable de administrar la tasa de ahorro de Dai. Permite a los usuarios depositar y retirar Dai, acumula los intereses generados y los transfiere al motor de deuda. El contrato mantiene un saldo de "Savings Dai" para cada usuario y el saldo total. Además, ofrece funciones para actualizar la tasa de ahorro y otras configuraciones. Facilita el ahorro de Dai y el crecimiento de los intereses para los usuarios en el ecosistema de MakerDAO.

## 🔍 Vat

In [ ]:
// SPDX-License-Identifier: AGPL-3.0-or-later

/// vat.sol -- Dai CDP database

// Copyright (C) 2018 Rain <rainbreak@riseup.net>
//
// This program is free software: you can redistribute it and/or modify
// it under the terms of the GNU Affero General Public License as published by
// the Free Software Foundation, either version 3 of the License, or
// (at your option) any later version.
//
// This program is distributed in the hope that it will be useful,
// but WITHOUT ANY WARRANTY; without even the implied warranty of
// MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
// GNU Affero General Public License for more details.
//
// You should have received a copy of the GNU Affero General Public License
// along with this program.  If not, see <https://www.gnu.org/licenses/>.

pragma solidity ^0.6.12;

// FIXME: This contract was altered compared to the production version.
// It doesn't use LibNote anymore.
// New deployments of this contract will need to include custom events (TO DO).

contract Vat {
    // --- Auth ---
    mapping (address => uint) public wards;
    function rely(address usr) external auth { require(live == 1, "Vat/not-live"); wards[usr] = 1; }
    function deny(address usr) external auth { require(live == 1, "Vat/not-live"); wards[usr] = 0; }
    modifier auth {
        require(wards[msg.sender] == 1, "Vat/not-authorized");
        _;
    }

    mapping(address => mapping (address => uint)) public can;
    function hope(address usr) external { can[msg.sender][usr] = 1; }
    function nope(address usr) external { can[msg.sender][usr] = 0; }
    function wish(address bit, address usr) internal view returns (bool) {
        return either(bit == usr, can[bit][usr] == 1);
    }

    // --- Data ---
    struct Ilk {
        uint256 Art;   // Total Normalised Debt     [wad]
        uint256 rate;  // Accumulated Rates         [ray]
        uint256 spot;  // Price with Safety Margin  [ray]
        uint256 line;  // Debt Ceiling              [rad]
        uint256 dust;  // Urn Debt Floor            [rad]
    }
    struct Urn {
        uint256 ink;   // Locked Collateral  [wad]
        uint256 art;   // Normalised Debt    [wad]
    }

    mapping (bytes32 => Ilk)                       public ilks;
    mapping (bytes32 => mapping (address => Urn )) public urns;
    mapping (bytes32 => mapping (address => uint)) public gem;  // [wad]
    mapping (address => uint256)                   public dai;  // [rad]
    mapping (address => uint256)                   public sin;  // [rad]

    uint256 public debt;  // Total Dai Issued    [rad]
    uint256 public vice;  // Total Unbacked Dai  [rad]
    uint256 public Line;  // Total Debt Ceiling  [rad]
    uint256 public live;  // Active Flag

    // --- Init ---
    constructor() public {
        wards[msg.sender] = 1;
        live = 1;
    }

    // --- Math ---
    function _add(uint x, int y) internal pure returns (uint z) {
        z = x + uint(y);
        require(y >= 0 || z <= x);
        require(y <= 0 || z >= x);
    }
    function _sub(uint x, int y) internal pure returns (uint z) {
        z = x - uint(y);
        require(y <= 0 || z <= x);
        require(y >= 0 || z >= x);
    }
    function _mul(uint x, int y) internal pure returns (int z) {
        z = int(x) * y;
        require(int(x) >= 0);
        require(y == 0 || z / y == int(x));
    }
    function _add(uint x, uint y) internal pure returns (uint z) {
        require((z = x + y) >= x);
    }
    function _sub(uint x, uint y) internal pure returns (uint z) {
        require((z = x - y) <= x);
    }
    function _mul(uint x, uint y) internal pure returns (uint z) {
        require(y == 0 || (z = x * y) / y == x);
    }

    // --- Administration ---
    function init(bytes32 ilk) external auth {
        require(ilks[ilk].rate == 0, "Vat/ilk-already-init");
        ilks[ilk].rate = 10 ** 27;
    }
    function file(bytes32 what, uint data) external auth {
        require(live == 1, "Vat/not-live");
        if (what == "Line") Line = data;
        else revert("Vat/file-unrecognized-param");
    }
    function file(bytes32 ilk, bytes32 what, uint data) external auth {
        require(live == 1, "Vat/not-live");
        if (what == "spot") ilks[ilk].spot = data;
        else if (what == "line") ilks[ilk].line = data;
        else if (what == "dust") ilks[ilk].dust = data;
        else revert("Vat/file-unrecognized-param");
    }
    function cage() external auth {
        live = 0;
    }

    // --- Fungibility ---
    function slip(bytes32 ilk, address usr, int256 wad) external auth {
        gem[ilk][usr] = _add(gem[ilk][usr], wad);
    }
    function flux(bytes32 ilk, address src, address dst, uint256 wad) external {
        require(wish(src, msg.sender), "Vat/not-allowed");
        gem[ilk][src] = _sub(gem[ilk][src], wad);
        gem[ilk][dst] = _add(gem[ilk][dst], wad);
    }
    function move(address src, address dst, uint256 rad) external {
        require(wish(src, msg.sender), "Vat/not-allowed");
        dai[src] = _sub(dai[src], rad);
        dai[dst] = _add(dai[dst], rad);
    }

    function either(bool x, bool y) internal pure returns (bool z) {
        assembly{ z := or(x, y)}
    }
    function both(bool x, bool y) internal pure returns (bool z) {
        assembly{ z := and(x, y)}
    }

    // --- CDP Manipulation ---
    function frob(bytes32 i, address u, address v, address w, int dink, int dart) external {
        // system is live
        require(live == 1, "Vat/not-live");

        Urn memory urn = urns[i][u];
        Ilk memory ilk = ilks[i];
        // ilk has been initialised
        require(ilk.rate != 0, "Vat/ilk-not-init");

        urn.ink = _add(urn.ink, dink);
        urn.art = _add(urn.art, dart);
        ilk.Art = _add(ilk.Art, dart);

        int dtab = _mul(ilk.rate, dart);
        uint tab = _mul(ilk.rate, urn.art);
        debt     = _add(debt, dtab);

        // either debt has decreased, or debt ceilings are not exceeded
        require(either(dart <= 0, both(_mul(ilk.Art, ilk.rate) <= ilk.line, debt <= Line)), "Vat/ceiling-exceeded");
        // urn is either less risky than before, or it is safe
        require(either(both(dart <= 0, dink >= 0), tab <= _mul(urn.ink, ilk.spot)), "Vat/not-safe");

        // urn is either more safe, or the owner consents
        require(either(both(dart <= 0, dink >= 0), wish(u, msg.sender)), "Vat/not-allowed-u");
        // collateral src consents
        require(either(dink <= 0, wish(v, msg.sender)), "Vat/not-allowed-v");
        // debt dst consents
        require(either(dart >= 0, wish(w, msg.sender)), "Vat/not-allowed-w");

        // urn has no debt, or a non-dusty amount
        require(either(urn.art == 0, tab >= ilk.dust), "Vat/dust");

        gem[i][v] = _sub(gem[i][v], dink);
        dai[w]    = _add(dai[w],    dtab);

        urns[i][u] = urn;
        ilks[i]    = ilk;
    }
    // --- CDP Fungibility ---
    function fork(bytes32 ilk, address src, address dst, int dink, int dart) external {
        Urn storage u = urns[ilk][src];
        Urn storage v = urns[ilk][dst];
        Ilk storage i = ilks[ilk];

        u.ink = _sub(u.ink, dink);
        u.art = _sub(u.art, dart);
        v.ink = _add(v.ink, dink);
        v.art = _add(v.art, dart);

        uint utab = _mul(u.art, i.rate);
        uint vtab = _mul(v.art, i.rate);

        // both sides consent
        require(both(wish(src, msg.sender), wish(dst, msg.sender)), "Vat/not-allowed");

        // both sides safe
        require(utab <= _mul(u.ink, i.spot), "Vat/not-safe-src");
        require(vtab <= _mul(v.ink, i.spot), "Vat/not-safe-dst");

        // both sides non-dusty
        require(either(utab >= i.dust, u.art == 0), "Vat/dust-src");
        require(either(vtab >= i.dust, v.art == 0), "Vat/dust-dst");
    }
    // --- CDP Confiscation ---
    function grab(bytes32 i, address u, address v, address w, int dink, int dart) external auth {
        Urn storage urn = urns[i][u];
        Ilk storage ilk = ilks[i];

        urn.ink = _add(urn.ink, dink);
        urn.art = _add(urn.art, dart);
        ilk.Art = _add(ilk.Art, dart);

        int dtab = _mul(ilk.rate, dart);

        gem[i][v] = _sub(gem[i][v], dink);
        sin[w]    = _sub(sin[w],    dtab);
        vice      = _sub(vice,      dtab);
    }

    // --- Settlement ---
    function heal(uint rad) external {
        address u = msg.sender;
        sin[u] = _sub(sin[u], rad);
        dai[u] = _sub(dai[u], rad);
        vice   = _sub(vice,   rad);
        debt   = _sub(debt,   rad);
    }
    function suck(address u, address v, uint rad) external auth {
        sin[u] = _add(sin[u], rad);
        dai[v] = _add(dai[v], rad);
        vice   = _add(vice,   rad);
        debt   = _add(debt,   rad);
    }

    // --- Rates ---
    function fold(bytes32 i, address u, int rate) external auth {
        require(live == 1, "Vat/not-live");
        Ilk storage ilk = ilks[i];
        ilk.rate = _add(ilk.rate, rate);
        int rad  = _mul(ilk.Art, rate);
        dai[u]   = _add(dai[u], rad);
        debt     = _add(debt,   rad);
    }
}

**Introduccion**

La función principal del VAT es  guardar los vaults y controlar los DAI asociados y el balance de los collateral. Además en VAT se definen las reglas en las cuales se pueden manipular los Vaults y los balances. Estas reglas son muy importantes y son consideradas la constitución del DSS (core system accounting).


**Mecanismos y conceptos**

El “vault” central es el de DAI y el estado del collateral es controlado en el Vat. El contrato de Vat no depende de nada externo y se encarga de que no cambie la contabilidad de DAI. En el Vat rigen una serie de principios que son:

1. No puede existir DAI sin collateral

  a) Ilk: un tipo de collateral

  b)Con la función slip, se modifica el balance del colateral de un usuario.

  c)flux Te permite transferir collateral entre usuarios



2. El vault de estructura de data (urn)
  - 'Ink' te dice el balance del colateral
  - Art te dice la deuda normalizada

3. Un colateral es un Ilk (ex: ETH-A)
  - Art: es la deuda normalizada de un ILK.
  - Rate: te dice el factor al que aumenta la deuda
  - Spot: precio con un margen de segurdad
  - Line: maximo de la deuda
  - Dust: te dice el minimo de la deuda

**Administración de los Vault**
- Los vaults administran a través de la función frob, que permite modificar vaults y emitir DAI.
- Los vaults son confiscados con la función grab, esta permite hacer las liquidaciones de las vaults.
- La función heal permite cancelar deuda confiscada.
DAI se puede transferir entre usuarios con la función move




# 🔍 Spot

In [ ]:
// SPDX-License-Identifier: AGPL-3.0-or-later

/// spot.sol -- Spotter

// This program is free software: you can redistribute it and/or modify
// it under the terms of the GNU Affero General Public License as published by
// the Free Software Foundation, either version 3 of the License, or
// (at your option) any later version.
//
// This program is distributed in the hope that it will be useful,
// but WITHOUT ANY WARRANTY; without even the implied warranty of
// MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
// GNU Affero General Public License for more details.
//
// You should have received a copy of the GNU Affero General Public License
// along with this program.  If not, see <https://www.gnu.org/licenses/>.

pragma solidity ^0.6.12;

// FIXME: This contract was altered compared to the production version.
// It doesn't use LibNote anymore.
// New deployments of this contract will need to include custom events (TO DO).

interface VatLike {
    function file(bytes32, bytes32, uint) external;
}

interface PipLike {
    function peek() external returns (bytes32, bool);
}

contract Spotter {
    // --- Auth ---
    mapping (address => uint) public wards;
    function rely(address guy) external auth { wards[guy] = 1;  }
    function deny(address guy) external auth { wards[guy] = 0; }
    modifier auth {
        require(wards[msg.sender] == 1, "Spotter/not-authorized");
        _;
    }

    // --- Data ---
    struct Ilk {
        PipLike pip;  // Price Feed
        uint256 mat;  // Liquidation ratio [ray]
    }

    mapping (bytes32 => Ilk) public ilks;

    VatLike public vat;  // CDP Engine
    uint256 public par;  // ref per dai [ray]

    uint256 public live;

    // --- Events ---
    event Poke(
      bytes32 ilk,
      bytes32 val,  // [wad]
      uint256 spot  // [ray]
    );

    // --- Init ---
    constructor(address vat_) public {
        wards[msg.sender] = 1;
        vat = VatLike(vat_);
        par = ONE;
        live = 1;
    }

    // --- Math ---
    uint constant ONE = 10 ** 27;

    function mul(uint x, uint y) internal pure returns (uint z) {
        require(y == 0 || (z = x * y) / y == x);
    }
    function rdiv(uint x, uint y) internal pure returns (uint z) {
        z = mul(x, ONE) / y;
    }

    // --- Administration ---
    function file(bytes32 ilk, bytes32 what, address pip_) external auth {
        require(live == 1, "Spotter/not-live");
        if (what == "pip") ilks[ilk].pip = PipLike(pip_);
        else revert("Spotter/file-unrecognized-param");
    }
    function file(bytes32 what, uint data) external auth {
        require(live == 1, "Spotter/not-live");
        if (what == "par") par = data;
        else revert("Spotter/file-unrecognized-param");
    }
    function file(bytes32 ilk, bytes32 what, uint data) external auth {
        require(live == 1, "Spotter/not-live");
        if (what == "mat") ilks[ilk].mat = data;
        else revert("Spotter/file-unrecognized-param");
    }

    // --- Update value ---
    function poke(bytes32 ilk) external {
        (bytes32 val, bool has) = ilks[ilk].pip.peek();
        uint256 spot = has ? rdiv(rdiv(mul(uint(val), 10 ** 9), par), ilks[ilk].mat) : 0;
        vat.file(ilk, "spot", spot);
        emit Poke(ilk, val, spot);
    }

    function cage() external auth {
        live = 0;
    }
}

El núcleo se compone de las siguientes partes:

1. Declaración de interfaces: declara las interfaces "VatLike" y "PipLike" que se utilizan para interactuar con otros contratos.

2. Declaración de variables de estado:
   - `wards`: Un mapeo que almacena los permisos de autorización para acceder a ciertas funciones.
   - `ilks`: Un mapeo que almacena los detalles de cada "ilk" (tipo de colateral), incluyendo el contrato del oráculo correspondiente y la relación de liquidación.
   - `vat`: Una variable que almacena la dirección del contrato "vat" que representa el núcleo del sistema MakerDAO.
   - `par`: Una variable que almacena el valor de DAI en el activo de referencia.
   - `live`: flag que indica si el contrato está activo o no.

3. Eventos: El contrato emite un evento llamado "Poke" cuando se llama a la función "poke".

4. Funciones administrativas:
   - `rely` y `deny`: Estas funciones permiten dar o revocar permisos de autorización a una dirección específica.
   - Funciones de modificación del estado del contrato: Estas funciones permiten actualizar los valores de las variables de estado, como los contratos de oráculo, la relación de liquidación y el valor de DAI.
   - `cage`: Esta función se utiliza para desactivar el contrato, estableciendo la variable `live` en 0.

5. Funciones de actualización del valor:
   - `poke`: actualiza el precio spot de un "ilk" específico consultando el oráculo correspondiente. Calcula el precio spot utilizando la relación de liquidación y actualiza el contrato "vat" con el nuevo precio.

6. Funciones matemáticas internas:
   - `mul`: Realiza una multiplicación segura que verifica si hay desbordamiento o subdesbordamiento.
   - `rdiv`: Hace una división segura y devuelve un número con la precisión adecuada.

Básicamente, se basa en mantener una lista de `lik` junto con su oráculo y relaciones de liquidación asociadas. Se encarga de actualizar los precios spot en el contrato `vat` utilizando la información proporcionada por los oráculos. Además, incluye funcionalidades administrativas para otorgar permisos y configurar variables clave.

# 👉 Obtener información del protocolo

## 📚 Instalo librerías:
Vamos a usar
- web3
- requests


In [ ]:
!pip install --upgrade ipython

In [ ]:
!pip install jedi>=0.16

In [ ]:
!pip install web3==5.31.3

In [ ]:
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install eth-brownie

## 🤓 Preguntas

Importamos las librerías previamente instaladas

In [ ]:
from web3 import Web3

In [ ]:
import requests

In [ ]:
def abis(address: str) -> str:
    address = str(address)
    _URL = 'https://api.etherscan.io/api'
    _KEY = 'TE6HWCVNEE8149WGFDSDS6U8WZTVPMEJ9Z'

    payload = {'module': 'contract',
               'action': 'getabi',
               'address': address.lower(),
               'apiKey': _KEY}

    req = requests.get(_URL, params=payload)
    res = req.json()

    return res['result']


Conexión a la blockchain mediante RPC

In [ ]:
rpc = "https://eth-mainnet.g.alchemy.com/v2/AZuYeL7x10B6NMTfUIyR2tFayr3Bl_rD"
web3 = Web3(Web3.HTTPProvider(rpc))

if web3.isConnected() == True:
        print('Conectado')
else:
        print('No conectado')

Conectado


Dirección de los contratos que vamos a utilizar, en este caso los de MakerDAO.

In [ ]:
daiAddress = '0x6B175474E89094C44Da98b954EedeAC495271d0F'
spotAddress = '0x65C79fcB50Ca1594B025960e539eD7A9a6D434A3'
vatAddress = '0x35D1b3F3D7966A1DFe207aa4514C12a259A0492B'
potAdress = '0x197E90f9FAD81970bA7976f33CbD77088E5D7cf7'
ilkAdress = '0x5a464C28D19848f44199D003BeF5ecc87d090F87'


daiContractABI = abis(daiAddress)

spotABI = abis(spotAddress)

vatContractABI = abis(vatAddress)

potABI = abis(potAdress)

ilkABI = abis(ilkAdress)


print(daiContractABI)
print(spotABI)
print(vatContractABI)
print(potABI)
print(ilkABI)
print()

[{"inputs":[{"internalType":"uint256","name":"chainId_","type":"uint256"}],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"src","type":"address"},{"indexed":true,"internalType":"address","name":"guy","type":"address"},{"indexed":false,"internalType":"uint256","name":"wad","type":"uint256"}],"name":"Approval","type":"event"},{"anonymous":true,"inputs":[{"indexed":true,"internalType":"bytes4","name":"sig","type":"bytes4"},{"indexed":true,"internalType":"address","name":"usr","type":"address"},{"indexed":true,"internalType":"bytes32","name":"arg1","type":"bytes32"},{"indexed":true,"internalType":"bytes32","name":"arg2","type":"bytes32"},{"indexed":false,"internalType":"bytes","name":"data","type":"bytes"}],"name":"LogNote","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"src","type":"address"},{"indexed":true,"internalType":"address","name":"dst",

In [ ]:
# Crea instancias de los contratos
daiContract = web3.eth.contract(address=daiAddress, abi=daiContractABI)
spotContract = web3.eth.contract(address=spotAddress, abi=spotABI)
vatContract = web3.eth.contract(address=vatAddress, abi=vatContractABI)
potContract = web3.eth.contract(address=potAdress, abi=potABI)
ilkRegistery = web3.eth.contract(address=ilkAdress, abi=ilkABI)

### ¿Qué tokens y su cantidad se encuentran depositados?

Para encontrar la cantidad de tokens, primero vamos a ver qué funciones tienen los contratos Vat y Spot.

In [ ]:
# Obtener una lista de las funciones disponibles en el contrato VAT
functionsVat = vatContract.functions
functionsSpot = spotContract.functions
functionsIlkRegistery = ilkRegistery.functions
# Imprimir la lista de funciones disponibles
print(f'Las funciones de Vat son: ' + str(dir(functionsVat)))
print(f'Las funciones de Spot son: ' + str(dir(functionsSpot)))
print(f'Las funciones de Ilk son: '+ str(dir(functionsIlkRegistery)))
# ilks está -> mapping of ilk type
# está urns -> mapping of urn (es un vault en específico)

Las funciones de Vat son: ['Line', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__gt__', '__hasattr__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_functions', 'abi', 'address', 'cage', 'can', 'dai', 'debt', 'deny', 'file', 'flux', 'fold', 'fork', 'frob', 'gem', 'grab', 'heal', 'hope', 'ilks', 'init', 'live', 'move', 'nope', 'rely', 'sin', 'slip', 'suck', 'urns', 'vice', 'wards', 'web3']
Las funciones de Spot son: ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__gt__', '__hasattr__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduc

Explorando los contratos, llegamos a ilk registry, que es donde están los Colaterales de Deuda Instantánea (Instant Loan Collateral). Los usuarios pueden bloquear activos como Ether (ETH), Basic Attention Token (BAT) y otros tokens en un smart contract llamado Contrato de Colateral de Deuda (CDP) y recibir a cambio una cantidad equivalente de Dai. Estos activos bloqueados se utilizan como garantía para asegurar la emisión de Dai y mantener su estabilidad.

Cada tipo de activo aceptado como colateral tiene un código de identificación único llamado ILK. Por ejemplo, "ETH-A" se refiere al ILK para Ether, "BAT-A" se refiere al ILK para Basic Attention Token, y así sucesivamente. Estos ILKs representan los diferentes tipos de colaterales aceptados y las condiciones específicas para utilizarlos.

Los listamos seguidamente:

In [ ]:
# Llamar a la función list() del contrato para obtener la lista de ilk types
ilk_types = ilkRegistery.functions.list().call()
ilks = []
# Imprimir los ilk types
for ilk_type in ilk_types:
    ilks.append(ilk_type)

ilks2 = [x.decode().strip('\x00') for x in ilks]
print('Los colaterales de deuda instantánea son: '+str(ilks2))

print('Es decir, hay un total de ' + str(len(ilks)) + ' de colaterales.')

Los colaterales de deuda instantánea son: ['ETH-A', 'ETH-B', 'ETH-C', 'BAT-A', 'USDC-A', 'USDC-B', 'WBTC-A', 'TUSD-A', 'ZRX-A', 'KNC-A', 'MANA-A', 'USDT-A', 'PAXUSD-A', 'COMP-A', 'LRC-A', 'LINK-A', 'BAL-A', 'YFI-A', 'GUSD-A', 'UNI-A', 'RENBTC-A', 'AAVE-A', 'UNIV2DAIETH-A', 'UNIV2WBTCETH-A', 'UNIV2USDCETH-A', 'UNIV2DAIUSDC-A', 'UNIV2ETHUSDT-A', 'UNIV2LINKETH-A', 'UNIV2UNIETH-A', 'UNIV2WBTCDAI-A', 'UNIV2AAVEETH-A', 'UNIV2DAIUSDT-A', 'PSM-USDC-A', 'RWA001-A', 'RWA002-A', 'RWA003-A', 'RWA004-A', 'RWA005-A', 'RWA006-A', 'PSM-PAX-A', 'MATIC-A', 'GUNIV3DAIUSDC1-A', 'WSTETH-A', 'RWA013-A', 'WBTC-B', 'WBTC-C', 'PSM-GUSD-A', 'GUNIV3DAIUSDC2-A', 'CRVV1ETHSTETH-A', 'WSTETH-B', 'RWA008-A', 'RWA009-A', 'TELEPORT-FW-A', 'RWA007-A', 'RETH-A', 'DIRECT-COMPV2-DAI', 'GNO-A', 'RWA010-A', 'RWA011-A', 'RWA012-A', 'DIRECT-AAVEV2-DAI', 'DIRECT-SPARK-DAI', 'RWA014-A']
Es decir, hay un total de 63 de colaterales.


In [ ]:
#Vat - debt
debt = (vatContract.functions.debt().call())
print(debt)
print('Debt es la suma de DAI emitido, hay un total de: ' + (str((debt) / 1000000000000000000 ) + ' DAI'))


4631826867594101274001073781410014253790015793568560817
Debt es la suma de DAI emitido, hay un total de: 4.631826867594101e+36 DAI


In [ ]:
# Vat - vice
print('Vice es la suma de todos los stablecoin sin respaldo, siendo un total de: ' + str(formatear_numero(vatContract.functions.vice().call())))

Vice es la suma de todos los stablecoin sin respaldo, siendo un total de: 23,638,153,486,194,599,044,099,651,589,664,045,433,107,345,387,290,624.000000000000000000


In [ ]:
# Info por cada colateral
ilks_info = ilkRegistery.functions.list().call()
for ilk in ilks:
    art = vatContract.functions.ilks(ilk).call()[0]
    rate = vatContract.functions.ilks(ilk).call()[1]
    spot = vatContract.functions.ilks(ilk).call()[2]
    line = vatContract.functions.ilks(ilk).call()[3]
    dust = vatContract.functions.ilks(ilk).call()[4]
    print(f"Ilk: {ilk}")
    print(f"Total de Deuda Normalizada (Art): {(art) / 1000000000000000000}")
    print(f"Tasas Acumuladas (rate): {rate}")
    print(f"Precio con Margen de Seguridad (spot): {spot}")
    print(f"Límite de Deuda (line): {line}")
    print(f"Piso de Deuda en la Urna (dust): {dust}")
    print()


Ilk: b'ETH-A\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
Total de Deuda Normalizada (Art): 178385170.70954895
Tasas Acumuladas (rate): 1090731669026522217994790656
Precio con Margen de Seguridad (spot): 1189582758620689655172413793103
Límite de Deuda (line): 344577738828264905694355614078499134968653609963882958
Piso de Deuda en la Urna (dust): 7500000000000000000000000000000000000000000000000

Ilk: b'ETH-B\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
Total de Deuda Normalizada (Art): 42956857.36455807
Tasas Acumuladas (rate): 1139363674158810634981112298
Precio con Margen de Seguridad (spot): 1326842307692307692307692307692
Límite de Deuda (line): 70285048201047811243736716224540185781454432616980706
Piso de Deuda en la Urna (dust): 25000000000000000000000000000000000000000000000000

Ilk: b'ETH-C\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

In [ ]:
total_art = 0

for ilk in ilks:
    art = vatContract.functions.ilks(ilk).call()[0]
    total_art += art

print(f"Total de Deuda Normalizada (Art) en todos los ilks: {total_art}")
print(total_art / 1000000000000000000) # 18 decimales


Total de Deuda Normalizada (Art) en todos los ilks: 4562753371710341146454964911
4562753371.710341


In [ ]:
total_line = 0

for ilk in ilks:
    line = vatContract.functions.ilks(ilk).call()[3]
    total_line += line

print(f"Límite de Deuda (Line) en todos los ilks: {total_line}")

print(total_line / 1000000000000000000) # 18 decimales
print(total_line / 10000000000) # 10 decimales


Límite de Deuda (Line) en todos los ilks: 5314400472124514949382207991293373503832129682846658937
5.314400472124515e+36
5.314400472124515e+44


**¿Qué tokens y su cantidad se encuentran depositados?**

Hay 63 distintos tipos de colaterales que respaldan los aproximadamente 4,627,827,169.61 de deuda en DAI. Estos colaterales están compuestos de tokens que pueden o no repetirse pero que difieren en ciertas reglas, como el stability fee, el liquidation ratio y el máximo de DAI que se pueden generar. Por ejemplo, en los colaterales ETH-A y en ETH-B se utiliza en ambos ETH como token de collateral pero, los liquidation fees, el stability y la cantidad máxima permitida de generación de DAI difieren.

Entonces, los tokens depositados como colateral en MakerDado son:

ETH, 'BAT, USDC, WBTC, TUSD, ZRX, KNC, MANA', 'USDT', 'PAXUSD', 'COMP', 'LRC', LINK, 'BAL, YFI, 'GUSD', 'UNI', 'RENBTC', 'AAVE', 'UNIV2DAIETH', 'UNIV2WBTCETH', 'UNIV2USDCETH, 'UNIV2DAIUSDC', 'UNIV2ETHUSDT’, 'UNIV2LINKETH', 'UNIV2UNIETH', 'UNIV2WBTCDAI', 'UNIV2AAVEETH', 'UNIV2DAIUSDT', 'PSM-USDC', 'RWA001', 'RWA002', 'RWA003', 'RWA004', 'RWA005', 'RWA006','RWA007,'RWA008', 'RWA009',’RWA010', 'RWA011', 'RWA012','RWA013',’RWA014’ 'PSM-PAX', 'MATIC-A', 'GUNIV3DAIUSDC1', 'WSTETH', ,'PSM-GUSD’, 'GUNIV3DAIUSDC2', 'CRVV1ETHSTETH',, 'TELEPORT-FW', ', 'RETH', 'DIRECT-COMPV2-DAI', 'GNO', ', 'DIRECT-AAVEV2-DAI', 'DIRECT-SPARK-DAI'.

La cantidad de cada uno de estos colaterales se encuentra en el código, representado con la variable ART, que muestra la cantidad de deuda normalizada de cada colateral. La cantidad de deuda normalizada es equivalente a la cantidad de DAI generado por cada colateral. La suma de toda la deuda normalizada de todos los colaterales debiera ser igual (o similar) a la cantidad total de deuda en DAI en circulación. Si vemos en el código podemos ver que los resultados obtenidos a través de la suma de los ART  de todos los ILK (colaterales) son muy parecidos al resultado obtenido de la cantidad total de DAI(deuda) emitida. Además, al compararlo con la página Daistats.com podemos respaldar que los resultados encontrados representan correctamente la cantidad de DAI emitido y por ende una aproximación muy buena de la cantidad de tokens depositados como colateral.


###  ¿Por qué creen que estos son los tokens depositados en estos protocolos?

La mayoría de los tokens depositados son:


*  ETH: Creemos que las razones por las que hay grandes cantidades de ETH depositado son: primero que el protocolo de Maker Dao funciona en la blockchain de Ethereum, lo que explica la gran cantidad de ethereum depositado y segundo que en sus comienzos, solo era posible depositar ETH como colateral para generar DAI.

*  WSTETH: wrapped st eth. St ETH es ETH stakeado en LIDO. Es uno de los colaterales ams utilizados. Poder emitir DAI con WSTETH te permite apalancarse aún más en protocolos DEFI.

*  RWA007: real world asset. Representa un activo del mundo real. Estos en general son activos de alta calidad que proveen un buen respaldo al DAI.

*  RWA014: real world asset.
*  PSM-USDC: este es un token que se adquiere a través del sistema PSM (peg stability module), un sistema dentro de Maker que te permite intercambiar DAI con USDC y otras stablecoins a un tipo de cambio fijo de 1:1.
*  PSM-USDC representa los USDC adquiridos a través del PSM. El propósito principal del
*  PSM-USDC es proporcionar una fuente de liquidez adicional y estabilizar el precio de DAI.

*  PSM-GUSD: lo mismo que PSM-USDC pero con la stablecoing GUSD.
*  PSM-USDP: lo mismo que PSM-USDC pero con la stablecoing GUSD.


### ¿Cambió esta estructura a lo largo del cambio de versiones?


## Sobre los cambios
La estructura experimentó varios cambios a lo largo de las versiones. Los más significativos fueron:

- Single-Collateral Dai (SCD): La primera versión de MakerDAO, conocida como Single-Collateral Dai, se basaba en un único tipo de colateral, Ethereum (ETH). Los usuarios podían "bloquear" ETH como garantía y generar Dai en función del valor de esa garantía.

- Multi-Collateral Dai (MCD): Con la actualización a Multi-Collateral Dai, se introdujo la capacidad de utilizar varios tipos de colateral para respaldar la emisión.
 - Ethereum (ETH): Ethereum fue el primer activo aceptado como colateral en MCD, al igual que en la versión anterior, SCD.
 - Basic Attention Token (BAT): BAT es el token de utilidad utilizado en el navegador Brave y se agregó como un colateral aceptado en MCD.
 - 0x (ZRX): ZRX es el token de protocolo utilizado en la plataforma 0x, que permite el intercambio de activos digitales descentralizado. Fue agregado como colateral en MCD.
 - Wrapped Bitcoin (WBTC): WBTC es una forma tokenizada de Bitcoin (BTC) que se ejecuta en la cadena de bloques de Ethereum. Fue agregado como un colateral aceptado en MCD.
 - USD Coin (USDC): USDC es una stablecoin vinculada al dólar estadounidense y se agregó como un colateral aceptado en MCD. Proporciona estabilidad al sistema y permite a los usuarios generar Dai respaldado por USDC.

- Introducción de Vaults: En las versiones más recientes, como MCD, se introdujo el concepto de "Vaults" o "Urns". Estos Vaults representan las cuentas individuales de los usuarios que depositan su colateral y generan Dai. Los Vaults proporcionan una mayor flexibilidad y control para los usuarios al gestionar su garantía y deuda.

- Gobernanza Descentralizada: también se trabajó en la descentralización de su gobernanza. Se introdujeron mecanismos como Maker Governance y MKR (el token de gobernanza de MakerDAO) para permitir a los participantes del ecosistema votar y tomar decisiones sobre aspectos clave del sistema.

- Actualizaciones y Mejoras: A lo largo de las versiones, se hicieron muchas actualizaciones y mejoras para abordar problemas de seguridad, eficiencia y escalabilidad. Estos cambios incluyeron modificaciones en los mecanismos de liquidación, ajustes en los parámetros de riesgo, implementación de oráculos mejorados y optimización de contratos inteligentes.
 - Inicialmente, se basaba en la red Ethereum (en la versión 1). Sin embargo, con el fin de mejorar la escalabilidad y reducir las tarifas de transacción, se hizo una transición a la red Ethereum 2.0 y adoptó otras redes blockchain.
 El cambio más notable fue la migración de Ethereum mainnet a Optimism ( es una solución de layer 2 que utiliza rollups para aumentar la escalabilidad y reducir las tarifas de transacción). Además, también transicionaron a Polygon, que también ofrece una infraestructura de layer 2 para transacciones rápidas y de bajo costo.

## Problemas
Hace unos meses, tras la caída de Silicon Valley Bank, se esperaba que haya cambios en cuánto a uno de los principales colaterales de DAI: USDC. En la actualidad, el 62 % de DAI se intercambia con USDC a través de PSM (Peg Stability Module) y menos del 20 % de DAI se acuña a través de criptomonedas. Esto hace que haya una tendencia a la centralización, y es probable que asuma el riesgo de USDC y USDT.

El 10 de marzo, Circle, el emisor de USDC, tenía una exposición al riesgo de 3.300 millones de dólares en Silicon Valley Bank. La quiebra de Silicon Valley Bank puede hacer que USDC no se pueda canjear por completo. El pánico en el mercado de las criptomonedas siguió aumentando y el USDC no estaba anclado, con el valor más bajo no anclado a $0,88. Huyó una gran cantidad de USDC, entre los cuales se cambiaron más de 700 millones de USDC por DAI a través de PSM y huyeron. Además, la mayor parte de la garantía de DAI también es USDC. El mercado también está en pánico por DAI, lo que llevó a la desvinculación de DAI. En la actualidad, USDC y DAI se han vuelto a anclar a 1 dólar, pero la preocupación de que DAI se parezca cada vez más a USDC no ha desaparecido. Cómo reducir el impacto de los riesgos de USDC y USDT en DAI es un problema en el que MakerDAO debe pensar.

Además, el USDC mantenido como garantía en las bóvedas de MakerDAO se utiliza para comprar bonos del Tesoro de EE. UU. En la actualidad, MakerDAO ha comprado 500 millones de bonos del Tesoro de EE. UU. a 6 meses y propuso asignar 750 millones de USDC adicionales para comprar bonos del Tesoro de EE. UU. el 8 de marzo para aumentar los ingresos del tesoro. Por un lado, esto ha convertido la mayor parte de la garantía de DAI en bonos del Tesoro de EE. UU., que están sujetos a agencias gubernamentales centralizadas y es probable que enfrenten un escrutinio regulatorio más estricto. Por otro lado, esto elimina el riesgo de USDC y aumenta los ingresos de MakerDAO.




### Cantidad de DAI en circulación y algunos datos desde el Frontend de Daistats




Estos son los datos encontrados a travez de daistats.com (front)

La cantidad de DAI en circulacion es de 4,808,016,856.44

El debt ceiling es de 6,620,668,903.49. Vendria a ser la canitdad maxima que se podria mintear con el collateral de hoy.

Los tokens depositados, utilizados como colateral  para genrar DAI son:
- RWA007 23.3%
- USDC 12.8%
- ETH 11.6%
- RWA014 10.4%
- USDP 10.4%
- GUSD 10.4%
- WSTETH 8.8%
- OTHERS 11.7%